### 모듈 임포트

In [ ]:
from ckonlpy.tag import Twitter, Postprocessor
from gensim.models import Word2Vec as w2v
import pandas as pd
import numpy as np

### 텍스트 파일 읽어오기

In [2]:
with open('./data/KCC940_Korean_sentences_UTF8.txt', 'r', encoding = 'utf-8') as text_file:
    sentence_list = text_file.readlines()

In [3]:
print('문장개수: ', len(sentence_list), '문장'), print('파일크기: ', 954231, 'KB')

문장개수:  6263454 문장
파일크기:  954231 KB


(None, None)

### 데이터프레임 변환 후 전처리

In [4]:
# 데이터프레임 변환
sentence_df = pd.DataFrame({'sentence': sentence_list})

In [5]:
# 결측치 확인
sentence_df.isnull().sum()

sentence    0
dtype: int64

In [6]:
# 데이터프레임 크기 확인
sentence_df.shape

(6263454, 1)

In [7]:
# 데이터프레임 각 행마다 앞 뒤 공백 제거
sentence_df['sentence'] = sentence_df['sentence'].str.strip()

In [8]:
# 정규 표현식 사용
sentence_df['sentence'] = sentence_df['sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [9]:
# 결측치가 아닌 공백을 결측치로 치환
sentence_df['sentence'].replace('', np.nan, inplace = True)

In [10]:
# 결측치 존재하는 행 제거, 해당 데이터에서는 결측치가 존재하지 않았음
sentence_df = sentence_df.dropna(axis = 0) 

In [11]:
# 불용어 정의
stopwords = {
    '의','가','이','은','들','는','좀','잘','걍','과',
    '도','를','으로','자','에','와','한','하다', '어요'
}

In [13]:
# ckonlpy Postprocessor value 정의, Twitter() 정의
passtags = {'Noun'}
twitter = Twitter()

postprocessor = Postprocessor(
    base_tagger = twitter, # base tagger
    stopwords = stopwords, # 해당 단어 필터
    #passwords = passwords, # 해당 단어만 선택
    passtags = passtags, # 해당 품사만 선택
    #replace = replace, # 해당 단어 set 치환
    #ngrams = ngrams # 해당 복합 단어 set을 한 단어로 결합
)

In [14]:
# 수정된 데이터프레임 리스트 변환
input_sentence_list = sentence_df['sentence'].to_list()

### 토큰화

In [ ]:
# 토큰화 진행
tokenized_nouns_list = []
i = 0
for sentence in input_sentence_list:
    tokenized_word = postprocessor.pos(sentence)
    for word in tokenized_word:
        tokenized_nouns_list.append(word[0])
    i += 1
    #print("{}/{}".format(i, len(input_sentence_list)))

In [ ]:
# 중복단어 체크
word_dict = {}
for i in tokenized_nouns_list:
    if i in word_dict:
        word_dict[i] += 1
    else: 
        word_dict[i] = 1
        
print(len(word_dict))

In [ ]:
# 빈도수 1~3인 단어 삭제
fixed_word_dict = {key: value for key, value in word_dict.items() if value != 1}
fixed_word_dict = {key: value for key, value in fixed_word_dict.items() if value != 2}
fixed_word_dict = {key: value for key, value in fixed_word_dict.items() if value != 3}
print(len(fixed_word_dict))

In [ ]:
# 데이터프레임 변환
nouns_list = list(fixed_word_dict.keys())
counts_list = list(fixed_word_dict.values())
counted_nouns_df = pd.DataFrame({'nouns': nouns_list, 'counts_list': counts_list})

In [ ]:
# count 기준 내림차순 정렬 및 인덱스 초기화
counted_nouns_df = counted_nouns_df.sort_values(by='counts_list', ascending=False).reset_index(drop = True)

In [ ]:
# 토큰화된 단어 빈도수 상위 10개 출력
counted_nouns_df[:10]

### 토큰 데이터 저장

In [ ]:
# 데이터프레임 CSV 형식 파일로 저장
counted_nouns_df.to_csv('./data/fixed_word_df.csv', index = False)